# Notebook 04: WRDS Data Processing

**Objective:**
1. Process raw daily CRSP stock data into monthly measures of **Crash Risk** and **Liquidity**.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
tqdm.pandas()

Load and clean daily data

In [3]:
file_path = 'data/crsp_daily_stock.csv'
cols_map = {
    'date': 'date',
    'PERMNO': 'PERMNO',
    'TICKER': 'TICKER',
    'CUSIP': 'CUSIP',
    'PRC': 'PRC',
    'VOL': 'VOL',
    'RET': 'RET',
    'SHROUT': 'SHROUT',
    'BID': 'BID',
    'ASK': 'ASK',
    'DLRET': 'DLRET',
    'vwretd': 'Mkt_Ret',
    'SHRCD': 'SHRCD',
    'EXCHCD': 'EXCHCD',
    'DLSTCD': 'DLSTCD'
}
df = pd.read_csv(
    file_path,
    usecols=cols_map.keys(),
    low_memory=False,
    dtype={
        'PERMNO': 'int32',
        'VOL': 'float32',
        'SHROUT': 'float32',
        'RET': 'object',
        'DLRET': 'object'
    }
)

df.rename(columns=cols_map, inplace=True)

df['date'] = pd.to_datetime(df['date'])

In [5]:
# 10/11 = Ordinary Common Shares (No ETFs, REITs, ADRs)
# 1/2/3 = NYSE, AMEX, NASDAQ (No OTC/Pink Sheets)
df = df[df['SHRCD'].isin([10, 11])]
df = df[df['EXCHCD'].isin([1, 2, 3])]

In [6]:
cols_to_numeric = ['RET', 'DLRET', 'PRC', 'VOL', 'BID', 'ASK', 'Mkt_Ret']
for c in cols_to_numeric:
    df[c] = pd.to_numeric(df[c], errors='coerce')

Calculate daily liquidity metrics

In [7]:
# handle negative prices
df['PRC'] = df['PRC'].abs()

# calculate total return
df['DLRET'] = df['DLRET'].fillna(0)
df['RET'] = df['RET'].fillna(0)
df['Total_Ret'] = ((1 + df['RET']) * (1 + df['DLRET'])) - 1

# handle missing market returns
df['Mkt_Ret'] = df['Mkt_Ret'].fillna(0)

prepare lags for price delay (Hou-Moskowitz)

In [8]:
# extract unique dates and market returns
market_data = df[['date', 'Mkt_Ret']].drop_duplicates().sort_values('date').set_index('date')

# create lags
for lag in range(1, 5):
    market_data[f'Mkt_Lag{lag}'] = market_data['Mkt_Ret'].shift(lag)

df = df.merge(market_data.drop(columns=['Mkt_Ret']), on='date', how='left')

In [9]:
# relative bid-ask spread
df['Midpoint'] = (df['ASK'] + df['BID']) / 2
df['Spread'] = np.where(df['Midpoint'] > 0, (df['ASK'] - df['BID']) / df['Midpoint'], np.nan)

# Amihud illiquidity
df['Dollar_Vol'] = df['PRC'] * df['VOL']
df['Amihud'] = (df['Total_Ret'].abs() / df['Dollar_Vol'].replace(0, np.nan)) * 1e6

# Turnover
df['Turnover'] = df['VOL'] / df['SHROUT']

Monthly Aggregation: Crash risk calculation

We group by ticker + month to calculate the skewness and volatitiy statistics

In [10]:
df['Month_ID'] = df['date'].dt.to_period('M')

In [11]:
def calculate_monthly_stats(group):
    # require 15 trading days
    if len(group) < 15:
        return pd.Series([np.nan]*7, index=['NCSKEW', 'DUVOL', 'Volatility', 'Avg_Spread', 'Avg_Amihud', 'Avg_Turnover', 'Price_Delay'])

    # we drop NA rows for the regression data
    reg_df = group[['Total_Ret', 'Mkt_Ret', 'Mkt_Lag1', 'Mkt_Lag2', 'Mkt_Lag3', 'Mkt_Lag4']].dropna()

    if len(reg_df) < 15:
        return pd.Series([np.nan]*7, index=['NCSKEW', 'DUVOL', 'Volatility', 'Avg_Spread', 'Avg_Amihud', 'Avg_Turnover', 'Price_Delay'])

    # extract Y (Firm Return) and X (Market + Lags)
    Y = reg_df['Total_Ret'].values

    # unrestricted matrix (Market + 4 Lags + Constant)
    # we verify all lags are finite
    X_unr = reg_df[['Mkt_Ret', 'Mkt_Lag1', 'Mkt_Lag2', 'Mkt_Lag3', 'Mkt_Lag4']].values
    X_unr = np.column_stack([np.ones(len(X_unr)), X_unr])

    # restricted matrix (Market Only + Constant)
    X_res = reg_df[['Mkt_Ret']].values
    X_res = np.column_stack([np.ones(len(X_res)), X_res])

    try:
        # unrestricted fit
        # lstsq returns: [coefs, sums_squared_residuals, rank, singular_values]
        beta_unr, resid_sum_sq_unr, _, _ = np.linalg.lstsq(X_unr, Y, rcond=None)

        # restricted Fit
        beta_res, resid_sum_sq_res, _, _ = np.linalg.lstsq(X_res, Y, rcond=None)

        # calculate R-squared: R2 = 1 - (SSR / SST)
        # SST = Sum of Squared Total Variations
        total_sum_sq = np.sum((Y - np.mean(Y))**2)

        # handle division by zero if SST is 0
        if total_sum_sq == 0:
            r2_unr = 0.0
            r2_res = 0.0
        else:
            # if lstsq returns empty residuals (perfect fit), sum is 0
            ssr_unr = resid_sum_sq_unr[0] if len(resid_sum_sq_unr) > 0 else 0.0
            ssr_res = resid_sum_sq_res[0] if len(resid_sum_sq_res) > 0 else 0.0

            r2_unr = 1 - (ssr_unr / total_sum_sq)
            r2_res = 1 - (ssr_res / total_sum_sq)

        # extract residuals for crash risk
        # we need the daily residuals from the unrestricted model
        # predicted Y = X @ Beta
        y_pred = X_unr @ beta_unr
        residuals = Y - y_pred

        # log-transform residuals: W = ln(1 + epsilon)
        # clip to avoid log of negative number
        w_ret = np.log(np.maximum(1 + residuals, 1e-8))

    except Exception:
        # catch singular matrix errors or other math faults
        return pd.Series([np.nan]*7, index=['NCSKEW', 'DUVOL', 'Volatility', 'Avg_Spread', 'Avg_Amihud', 'Avg_Turnover', 'Price_Delay'])

    # crash Risk (calculated on 'W')
    # NCSKEW
    # manual Skewness
    n = len(w_ret)
    w_mean = np.mean(w_ret)
    w_std = np.std(w_ret, ddof=1)

    if w_std == 0:
        ncskew = np.nan
        duvol = np.nan
    else:
        # skewness formula
        skew = (np.sum((w_ret - w_mean)**3) / n) / (w_std**3)
        # adjustment for sample skewness (Fisher-Pearson)
        if n > 2:
            skew_unbiased = skew * np.sqrt(n*(n-1)) / (n-2)
            ncskew = -1 * skew_unbiased
        else:
            ncskew = np.nan

        # DUVOL
        down_w = w_ret[w_ret < w_mean]
        up_w = w_ret[w_ret >= w_mean]
        n_down = len(down_w)
        n_up = len(up_w)

        if len(down_w) > 1 and len(up_w) > 1:
            ssr_down = np.sum((down_w - w_mean)**2)
            ssr_up = np.sum((up_w - w_mean)**2)

            duvol = np.log((ssr_down / (n_down - 1)) / (ssr_up / (n_up - 1)))
        else:
            duvol = np.nan

    # volatility
    vol = np.std(group['Total_Ret'], ddof=1) * np.sqrt(252)

    # liquidity averages
    avg_spread = group['Spread'].mean()
    avg_amihud = group['Amihud'].mean()
    avg_turnover = group['Turnover'].mean()

    # price efficiency
    price_delay = np.nan
    if r2_unr > 1e-4:
        ratio = min(r2_res / r2_unr, 1.0)
        price_delay = 1 - ratio
    else:
        price_delay = 0

    return pd.Series([ncskew, duvol, vol, avg_spread, avg_amihud, avg_turnover, price_delay],
                     index=['NCSKEW', 'DUVOL', 'Volatility', 'Avg_Spread', 'Avg_Amihud', 'Avg_Turnover', 'Price_Delay'])

In [12]:
# group values
df_monthly = df.groupby(['TICKER', 'CUSIP', 'Month_ID']).progress_apply(calculate_monthly_stats, include_groups=False).reset_index()

# clean values
cols_to_clean = ['NCSKEW', 'DUVOL', 'Avg_Spread', 'Avg_Amihud', 'Price_Delay']
df_monthly[cols_to_clean] = df_monthly[cols_to_clean].replace([np.inf, -np.inf], np.nan)

100%|██████████| 590284/590284 [11:15<00:00, 873.48it/s] 


In [13]:
output_path = 'data/monthly_outcomes.csv'
df_monthly.to_csv(output_path, index=False)